In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import re
import copy
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, classification_report

In [ ]:
df1 = pd.read_csv("Datasets/instagram_reach.csv")
df1.sample()

In [ ]:
df1.reset_index(drop=True)
df1["Time"] = df1["Time since posted"].apply(lambda x: int(x.split(" ")[0]))
df1.drop(columns=["Unnamed: 0","S.No", "Caption", "USERNAME","Time since posted"], inplace=True)

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
print(df1.loc[:0]["Hashtags"])

In [ ]:
def convert_hastag(text):
    ps = PorterStemmer()
    wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    corpus = []
    tag = re.sub("[^a-zA-Z]", " ", text)
    tag = tag.lower()
    tag = tag.split()
#     tag = [wordnet.lemmatize(word) for word in tag if not word in set(stopwords.words("english"))]
    
    return tag

convert_hastag(df1["Hashtags"][0])

In [ ]:
hastag = pd.DataFrame()
df1["Hashtags"]= df1["Hashtags"].apply(convert_hastag)

In [ ]:
print(df1[["Hashtags"]])

In [ ]:
unique_hashtag = set(tag for row in df1["Hashtags"] for tag in row)
len(unique_hashtag)

In [ ]:
for hashtag in unique_hashtag:
    df1[hashtag] = 0 # initialize 0 value for every hashtag


In [ ]:
for index, row in enumerate(df1["Hashtags"]): # enumerate hashtags 
    for tag in row: # Row wise hashtag 
        df1.at[index, tag] = 1 # appply 1 value when index and tag match 


In [ ]:
df1.drop(columns=["Hashtags"], inplace=True)

In [ ]:
x1 = df1.drop(columns=["Likes", "Time"])

In [ ]:
y1 = df1[["Likes", "Time"]]

In [ ]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.1)

In [ ]:
print(f"X_train shape = {x1_train.shape}")
print(f"X_test shape = {x1_test.shape}")
print(f"y_train shape = {y1_train.shape}")
print(f"y_test shape = {y1_test.shape}")

In [ ]:
y1_test

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(x1_train, y1_train)

In [ ]:
y1_pred = rf.predict(x1_test)
y1_pred

In [ ]:
mse1 = mean_squared_error(y1_test, y1_pred)
mse1

In [ ]:
rmse1 = np.sqrt(mse1)
rmse1

In [ ]:
r2score1 = r2_score(y1_test, y1_pred)
r2score1

In [ ]:
df2 = pd.read_csv("DataSets/ObesityDataSet_raw_and_data_sinthetic.csv")

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.describe()

In [ ]:
df2.info()

In [ ]:
df2.sample(5)

In [ ]:
df2["SCC"].unique()

In [ ]:
df2[["Age", "Weight"]] = df2[["Age", "Weight"]].astype(int)

In [ ]:
df2["Gender"] = df2["Gender"].apply(lambda x : 1 if x == "Male" else 0)

In [ ]:
df2["SMOKE"] = df2["SMOKE"].apply(lambda x : 1 if x == "yes" else 0)

In [ ]:
df2["family_history_with_overweight"] = df2["family_history_with_overweight"].apply(lambda x : 1 if x == "yes" else 0)

In [ ]:
df2["FAVC"] = df2["FAVC"].apply(lambda x : 1 if x == "yes" else 0)

In [ ]:
df2["SCC"] = df2["SCC"].apply(lambda x : 1 if x == "yes" else 0)

In [ ]:
df2

In [ ]:
df2["MTRANS"].unique()

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df2["CAEC"] =label_encoder.fit_transform(df2["CAEC"])

In [ ]:
df2["CALC"] = label_encoder.fit_transform(df2["CALC"])

In [ ]:
df2["MTRANS"] = label_encoder.fit_transform(df2["MTRANS"])

In [ ]:
df2["NObeyesdad"] = label_encoder.fit_transform(df2["NObeyesdad"])

In [ ]:
df2.sample(5)

In [ ]:
df2.info()

# 4 Different Classification algorithms

In [ ]:
x2 = df2.drop(columns=["NObeyesdad"])
y2 = df2["NObeyesdad"]

In [ ]:
y2

In [ ]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2, test_size=0.2)

In [ ]:
log_r2 = LogisticRegression(solver="liblinear")

In [ ]:
log_r2.fit(x2_train, y2_train)

In [ ]:
y2_pred_log = log_r2.predict(x2_test)

In [ ]:
accuracy_score(y2_test, y2_pred_log)

In [ ]:
knn2 = KNeighborsClassifier()

In [ ]:
knn2.fit(x2_train, y2_train)

In [ ]:
y2_pred_knn = knn2.predict(x2_test)

In [ ]:
accuracy_score(y2_test, y2_pred_knn)

In [ ]:
svm2 = svm.SVC(kernel="linear")

In [ ]:
svm2.fit(x2_train, y2_train)

In [ ]:
y2_pred_svm = svm2.predict(x2_test)

In [ ]:
accuracy_score(y2_test, y2_pred_svm)

In [ ]:
naive2 = MultinomialNB()

In [ ]:
naive2.fit(x2_train, y2_train)

In [ ]:
y2_pred_naive = naive2.predict(x2_test)

In [ ]:
accuracy_score(y2_test, y2_pred_naive)

In [14]:
df_3 =  pd.read_json("DataSets/News_Category_Dataset_v3.json", lines=True)

In [15]:
df_3.shape

(209527, 6)

In [16]:
df_3_new = copy.deepcopy(df_3)

In [17]:
df_3_new = df_3.iloc[:10000]

In [18]:
df_3_new.shape

(5000, 6)

In [19]:
def link_filter(text):
    replace_text = text.replace("https://www.huffpost.com/entry/", "")
    replace_text = replace_text.replace("https://www.huffingtonpost.com/entry/", "")
    replace_text = replace_text.split("_")[0]
    replace_text = re.sub("[^a-zA-Z]", " ",replace_text )
    return replace_text

In [20]:
df_3_new["link"]= df_3["link"].apply(link_filter)

<ipython-input-20-5734f4b261ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["link"]= df_3["link"].apply(link_filter)


In [21]:
df_3_new.shape

(5000, 6)

In [22]:
def text_preprecess(text, use_stop_word=True):
    ps = PorterStemmer()
#     wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    tag = re.sub("[^a-zA-Z]", " ", text)
    tag = tag.lower()
    tag = tag.split()
    tag = [ps.stem(word) for word in tag if not word in set(stopwords.words("english"))] if use_stop_word else tag
    
    return tag

In [23]:
df_3_new["link"] = df_3_new["link"].apply(text_preprecess)

<ipython-input-23-becf598e9aaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["link"] = df_3_new["link"].apply(text_preprecess)


In [24]:
df_3_new["headline"] = df_3_new["headline"].apply(text_preprecess)

<ipython-input-24-aae1c0916775>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["headline"] = df_3_new["headline"].apply(text_preprecess)


In [25]:
df_3_new["short_description"] = df_3_new["short_description"].apply(text_preprecess)

<ipython-input-25-2e5528fe1e7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["short_description"] = df_3_new["short_description"].apply(text_preprecess)


In [26]:
df_3_new["authors"] = df_3_new["authors"].apply(text_preprecess, use_stop_word=False)

<ipython-input-26-c1fe2bb1bb6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["authors"] = df_3_new["authors"].apply(text_preprecess, use_stop_word=False)


In [27]:
df_3_new["category"] = df_3_new["category"].apply(text_preprecess, use_stop_word=False)

<ipython-input-27-ade343f7bd2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["category"] = df_3_new["category"].apply(text_preprecess, use_stop_word=False)


In [28]:
df_3_new["tags"] = df_3_new["link"] + df_3_new["headline"] + df_3_new["short_description"] + df_3_new["category"] + df_3_new["authors"]

<ipython-input-28-959926ff48fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["tags"] = df_3_new["link"] + df_3_new["headline"] + df_3_new["short_description"] + df_3_new["category"] + df_3_new["authors"]


In [29]:
df_3_new.drop(columns=["link", "headline", "short_description", "category", "authors","date"], inplace=True)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
df_3_new["tags"] = df_3_new["tags"].apply(lambda text : " ".join(text))

<ipython-input-30-3e48c611d9f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_new["tags"] = df_3_new["tags"].apply(lambda text : " ".join(text))


In [31]:
df_3_new.shape

(5000, 1)

In [32]:
df_3_new

,tags
0,covid booster uptak us million american roll s...
1,american airlin passeng ban flight attend punc...
2,funniest tweet cat dog septemb funniest tweet ...
3,funniest parent tweet funniest tweet parent we...
4,ami cooper lose discrimin lawsuit franklin tem...
...,...
4995,anthoni causi dead new york post photograph an...
4996,oliv veronesi need coor light year old woman g...
4997,donald trump fire fauci twitter trump retweet ...
4998,war pandem coronaviru covid pandem war reli ti...


In [37]:
cv3 = CountVectorizer(max_features=5000)

In [38]:
vector = cv3.fit_transform(df_3_new["tags"]).toarray()

In [45]:
vector[0].shape

(5000,)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [41]:
similarity[5]

array([0.04883863, 0.02248469, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [42]:
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

# Function to find the most similar data point using Euclidean distance
def find_most_similar_data(dataset, given_data):
    distances = [euclidean_distance(given_data, data) for data in dataset]
    most_similar_index = np.argmin(distances)
    return dataset[most_similar_index]

In [47]:
# Example usage:
given_data = vector[0]  # The point for which we want to find the most similar point
most_similar_data = find_most_similar_data(vector, given_data)

print("Given data point:", given_data)
print("Most similar data point:", most_similar_data)

Given data point: [0 0 0 ... 0 0 0]
Most similar data point: [0 0 0 ... 0 0 0]


In [ ]:
#Jaccard Similarity: